In [ ]:
# Import packages
import os
import re  # regular expressions
import warnings
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma
import rasterio as rio
from rasterio.plot import plotting_extent
import geopandas as gpd
import earthpy as et
import earthpy.plot as ep
import earthpy.spatial as es
import earthpy.mask as em

import rioxarray as rxr

from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint

from sklearn.datasets import dump_svmlight_file

from glob import glob

from queue import Queue

warnings.simplefilter('ignore')

# Set working directory
os.chdir(os.path.join(et.io.HOME, 'BD', 'BA_DATA'))

In [ ]:
# Constants
d_limit = 1200

In [ ]:
vege_fs84 = glob('VegeData_16day/*h08v04*')

In [ ]:
vege_fs84

In [ ]:
vege_fs85 = glob('VegeData_16day/*h08v05*')

In [ ]:
vege_fs85

In [ ]:
vege_path = os.path.join("VegeData_16day", "MOD13A2.A2000049.h08v05.006.2015136104428.hdf")

In [ ]:
tem_path = os.path.join("TemData", "MOD11A2.A2000049.h08v05.006.2015058135046.hdf")

### Mask function

Pick any location, divide the neighborhood into 10 regions of interest in various directions, each region is within 50 km, 36 degree and 50 km sector as a region. 

In [ ]:
def sector_mask(shape,centre,radius,angle_range):
    """
    Return a boolean mask for a circular sector. The start/stop angles in  
    `angle_range` should be given in clockwise order.
    """

    x,y = np.ogrid[:shape[0],:shape[1]]
    cx,cy = centre
    tmin,tmax = np.deg2rad(angle_range)

    # ensure stop angle > start angle
    if tmax < tmin:
            tmax += 2*np.pi

    # convert cartesian --> polar coordinates
    r2 = (x-cx)*(x-cx) + (y-cy)*(y-cy)
    theta = np.arctan2(x-cx,y-cy) - tmin

    # wrap angles between 0 and 2*pi
    theta %= (2*np.pi)

    # circular mask
    circmask = r2 <= radius*radius

    # angular mask
    anglemask = theta <= (tmax-tmin)

    return circmask*anglemask

Examples of using "sector_mask"

In [ ]:
vege_modis[0]

In [ ]:
vege_modis.shape

In [ ]:
vege_matrix = vege_modis.reshape(1200,1200)

In [ ]:
vege_matrix.shape

In [ ]:
vege_matrix

In [ ]:
mask = sector_mask(vege_matrix.shape, (400, 600), 50, (0,36)) # need to consider the boundary cases

In [ ]:
from matplotlib import pyplot as pp

In [ ]:
pp.imshow(vege_matrix)
pp.show()

In [ ]:
test_matrix = vege_matrix.copy()
test_matrix[~mask] = -3000
pp.imshow(test_matrix)
pp.show()

In [ ]:
test_matrix.shape

In [ ]:
test_matrix[mask]

In [ ]:
test_matrix[mask].shape

In [ ]:
a = test_matrix[mask]

In [ ]:
a[:][a[:] == -3000] = -2000

In [ ]:
np.mean(a)

In [ ]:
a

In [ ]:
b.shape

In [ ]:
pp.imshow(vege_matrix)
pp.show()

### Process 16-day hdf file

16-day for vegetation_data

In [ ]:
def compute_feature_set(matrix, fill_value, new_value, row, colmn):
    """
    Return a feature set of a single pixel with row number r and column number c in the 1200 * 1200 matrix MATRIX.
    Feature set is computed based on the average value in 10 regions in one time frame for the feature SDS from FILENAMES.
    Fill_value will be replaced by new_value in the feature matrix.
    """
    feature_set = []
    for i in range(10):
        mask = sector_mask(vege_matrix.shape, (row, column), radius, (i, i + 36))
        sector = vege_matrix[mask]
        sector[:][sector[:] == fill_value] = new_value
        mean_value = np.mean(sector)
        feature_set.append(mean_value)
    return feature_set

In [ ]:
def vegetation_features(file, sds, fill_value, new_value, radius):
    """
    Return feature sets of all pixels in the file FILE.
    SDS is NDVI or EVI.
    
    """
    all_bands = []
    with rio.open(file) as dataset:
        for name in dataset.subdatasets:
            if re.search(sds, name):
                with rio.open(name) as subdataset:
                    modis_meta = subdataset.profile
                    all_bands.append(subdataset.read(1))
    vege_modis = np.stack(all_bands)
    vege_matrix = vege_modis[0].reshape(d_limit,d_limit)
    feature_sets = np.zeros((d_limit, d_limit))
    for r in range(d_limit):
        for c in range(d_limit):
            temp = compute_feature_set(vege_martrix, fill_value, new_value, r, c)
            feature_sets[r][c] = temp

Examples of using "compute_feature_set"

In [ ]:
aa = compute_feature_set(vege_fs85[0], "NDVI", -3000, -2000, 400, 600, 50)

In [ ]:
aa

In [ ]:
bb = compute_feature_set(vege_fs85[0], "NDVI", -3000, -2000, 100, 600, 50)

In [ ]:
bb

### Reduce dimension

In [ ]:
def rebin(m, shape):
    """
    Reshape the input matrix A to the shape SHAPE.
    """
    sh = shape[0],m.shape[0]//shape[0],shape[1],m.shape[1]//shape[1]
    return m.reshape(sh).mean(-1).mean(1)

### Process 8-day files

8-day for thermal_data, tem_data and NE_data

In [ ]:
def compute_features_8day(filenames, num_file, sds, fill_value, new_value, radius):
    """
    Combine every two 8-day files to compute feature sets. There are in total NUM_FILES in filenames. Used for 8-day datasets.
    Use a queue to help iterate through every two files.
    Input filenames is the result from using glob command
    Result is a 3d np array with all the SDS features using 10-region method
    """
    file_count = num_file / 2
    
    all_feature_sets = np.zeros((file_count, d_limit, d_limit))
    
    q = Queue(maxsize = num_file)
    
    for filename in filenames:
        queue.put(filename)
        
    for i in range (file_count):
        f1 = q.get()
        f2 = q.get()
        f1_bands = []
        f2_bands = []
        # open two files in a round
        with rio.open(f1) as dataset:
            for name in dataset.subdatasets:
                if re.search(sds, name):
                    with rio.open(name) as subdataset:
                        modis_meta = subdataset.profile
                        all_bands.append(subdataset.read(1))
        f1_modis = np.stack(f1_bands)
        f1_matrix = vege_modis[0].reshape(d_limit,d_limit)
        with rio.open(f2) as dataset:
            for name in dataset.subdatasets:
                if re.search(sds, name):
                    with rio.open(name) as subdataset:
                        modis_meta = subdataset.profile
                        all_bands.append(subdataset.read(1))
        f2_modis = np.stack(f2_bands)
        f2_matrix = vege_modis[0].reshape(d_limit,d_limit)
        # combine two matrices
        combined_matrix = (f1_matrix + f2_matrix) / 2
        feature_sets = np.zeros((d_limit, d_limit))
        for r in range(d_limit):
            for c in range(d_limit):
                temp = compute_feature_set(combined_matrix, fill_value, new_value, r, c)
                feature_sets[r][c] = temp
        
        all_feature_sets[i] = feature_sets
        return all_feature_sets

Within a region, exact a small set of features, and ask is there any fire observed in the region in the past 16 days. Consider the intensity of fire, number of pixels with fire (a pixel with fire for 3 days are considered as 3 fire), for a region in the past 16 days.